In [2]:
%load_ext autoreload
%autoreload 2

UsageError: Line magic function `%` not found.


In [3]:
import pandas as pd

from util.prediction import prepare_experiment_dfs, ModelEvaluator, split_train_test

df_crawled, df_reg, df_editreg, df_class = prepare_experiment_dfs('data/events/all_events.csv.gz')

In [4]:
# Perform gridsearch
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

df_train_full, df_test = split_train_test(df_class, pd.to_datetime('2016-01-01'), 45, 12)
param_grid_xgboost_reg = {'max_depth': [4, 5, 6, 10, 15, 20],
                          'n_estimators': [10, 100, 500, 1000],
                          'learning_rate': [1, 0.5, 0.1, 0.01, 0.001],
                          'gamma': [0, 1, 2, 5, 10],
                          'reg_lambda': [0, 1, 10],
                          'scale_pos_weight': [0.05, 0.1, 1, 2]}
param_grid_xgboost_class_viewed = {'max_depth': [4, 5, 6, 10, 15, 20],
                                   'n_estimators': [10, 100, 500, 1000],
                                   'learning_rate': [0.5, 0.3, 0.1, 0.01, 0.001],
                                   'gamma': [0, 1, 2, 5, 10],
                                   'reg_lambda': [0.1, 1, 10],
                                   'colsample_bytree': [0.1, 0.25, 0.5, 0.75, 1],
                                   'scale_pos_weight': [1,
                                                        sum(df_train_full.noticed == 0) / sum(
                                                            df_train_full.noticed == 1)],
                                   'subsample': [0.1, 0.25, 0.5, 0.75, 1]
                                   }
param_grid_xgboost_class_edited = {'max_depth': [4, 5, 6, 10, 15, 20],
                                   'n_estimators': [10, 100, 500, 1000],
                                   'learning_rate': [0.5, 0.3, 0.1, 0.01, 0.001],
                                   'gamma': [0, 1, 2, 5, 10],
                                   'reg_lambda': [0.1, 1, 10],
                                   'colsample_bytree': [0.1, 0.25, 0.5, 0.75, 1],
                                   'scale_pos_weight': [1,
                                                        sum(df_train_full.edited == 0) / sum(
                                                            df_train_full.edited == 1)],
                                   'subsample': [0.1, 0.25, 0.5, 0.75, 1]
                                   }

param_grid_rf = {'max_depth': [5, 10, 50, 100, None],
                 'n_estimators': [10, 100, 500, 1000],
                 'max_features': ['log2', 'sqrt'],
                 'min_samples_split': [2, 5, 10],
                 'bootstrap': [True, False],
                 'min_samples_leaf': [1, 2, 5]}
param_grid_svc = {'C': [0.1, 1, 10, 100],
                  'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
                  'degree': [1, 2, 3, 5],
                  'gamma': ['scale', 'auto'],
                  'class_weight': [None, 'balanced']}
param_grid_svr = {'C': [0.1, 1, 10, 100],
                  'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
                  'degree': [1, 2, 3, 5],
                  'gamma': ['scale', 'auto'],
                  'epsilon': [0, 0.01, 0.1, 1]}

param_grid_negative_binomial = None  # needs no grid search

# Perform hyperparameter search

In [6]:
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor
import pickle

In [7]:
def save_object(obj, filename, path='crossval_results'):
    if obj.gs_res:
        try:
            if obj.gs_res.best_estimator_:
                with open(f'{path}/{filename}.pkl', 'wb') as outp:  # Overwrites any existing file.
                    pickle.dump(obj.gs_res, outp, pickle.HIGHEST_PROTOCOL)
        except:
            print(f'No estimator trained for {filename}. Not saving')
    else:
        print(f'No estimator trained for {filename}. Not saving')

In [ ]:
n_jobs = 162
n_iter = 1000

# ================ XGB ========================
# noticed
grid_xgb_viewed_noreg = ModelEvaluator(XGBClassifier(n_jobs=n_jobs), df_class, 'noticed', 'f1_micro',
                                       params=param_grid_xgboost_class_viewed, factor_cols=['code', 'cat'])
grid_xgb_viewed_noreg.random_search(n_jobs=1, n_iter=n_iter)
grid_xgb_viewed_noreg.gs_res = grid_xgb_viewed_noreg.rs_res
save_object(grid_xgb_viewed_noreg, 'noreg_viewed_xgb')

# edited
grid_xgb_edit_noreg = ModelEvaluator(XGBClassifier(n_jobs=n_jobs), df_class, 'edited', 'f1_micro',
                                     params=param_grid_xgboost_class_edited, factor_cols=['code', 'cat'])
grid_xgb_edit_noreg.random_search(n_jobs=1, n_iter=n_iter)
grid_xgb_edit_noreg.gs_res = grid_xgb_edit_noreg.rs_res
save_object(grid_xgb_edit_noreg, 'noreg_edited_xgb')

# views
grid_xgb_views7sum_noreg = ModelEvaluator(XGBRegressor(n_jobs=n_jobs), df_reg, 'views_7_sum_log',
                                          'neg_mean_squared_error', params=param_grid_xgboost_reg,
                                          factor_cols=['code', 'cat'])
grid_xgb_views7sum_noreg.random_search(n_jobs=1, n_iter=n_iter)
grid_xgb_views7sum_noreg.gs_res = grid_xgb_views7sum_noreg.rs_res
save_object(grid_xgb_views7sum_noreg, 'noreg_views_xgb')

# edits
grid_xgb_edits7sum_noreg = ModelEvaluator(XGBRegressor(n_jobs=n_jobs), df_editreg, 'edits_7_sum_log',
                                          'neg_mean_squared_error', params=param_grid_xgboost_reg,
                                          factor_cols=['code', 'cat'])
grid_xgb_edits7sum_noreg.random_search(n_jobs=1, n_iter=n_iter)
grid_xgb_edits7sum_noreg.gs_res = grid_xgb_edits7sum_noreg.rs_res
save_object(grid_xgb_edits7sum_noreg, 'noreg_edits_xgb')

# ================ RF ========================
grid_rf_edited_noreg = ModelEvaluator(RandomForestClassifier(n_jobs=-1), df_class, 'edited', 'f1_micro',
                                      params=param_grid_rf, factor_cols=['code', 'cat'])
grid_rf_edits7sum_noreg = ModelEvaluator(RandomForestRegressor(n_jobs=-1), df_editreg, 'edits_7_sum_log',
                                         'neg_mean_squared_error', params=param_grid_rf, factor_cols=['code', 'cat'])
grid_rf_viewed_noreg = ModelEvaluator(RandomForestClassifier(n_jobs=-1), df_class, 'noticed', 'f1_micro',
                                      params=param_grid_rf, factor_cols=['code', 'cat'])
grid_rf_views7sum_noreg = ModelEvaluator(RandomForestRegressor(n_jobs=-1), df_reg, 'views_7_sum_log',
                                         'neg_mean_squared_error', params=param_grid_rf, factor_cols=['code', 'cat'])

grid_rf_viewed_noreg.random_search(n_jobs=-1, n_iter=n_iter)
grid_rf_viewed_noreg.gs_res = grid_rf_viewed_noreg.rs_res
save_object(grid_rf_viewed_noreg, 'noreg_viewed_rf')

grid_rf_edited_noreg.random_search(n_jobs=-1, n_iter=n_iter)
grid_rf_edited_noreg.gs_res = grid_rf_edited_noreg.rs_res
save_object(grid_rf_edited_noreg, 'noreg_edited_rf')

grid_rf_views7sum_noreg.random_search(n_jobs=-1, n_iter=n_iter)
grid_rf_views7sum_noreg.gs_res = grid_rf_views7sum_noreg.rs_res
save_object(grid_rf_views7sum_noreg, 'noreg_views_rf')

grid_rf_edits7sum_noreg.random_search(n_jobs=-1, n_iter=n_iter)
grid_rf_edits7sum_noreg.gs_res = grid_rf_edits7sum_noreg.rs_res
save_object(grid_rf_edits7sum_noreg, 'noreg_edits_rf')

# ================ SVC ========================
grid_svc_views7sum_noreg = ModelEvaluator(SVR(), df_reg, 'views_7_sum_log', 'neg_mean_squared_error',
                                          params=param_grid_svr, factor_cols=['code', 'cat'])
grid_svc_edits7sum_noreg = ModelEvaluator(SVR(), df_reg, 'edits_7_sum_log', 'neg_mean_squared_error',
                                          params=param_grid_svr, factor_cols=['code', 'cat'])
grid_svc_edit_noreg = ModelEvaluator(SVC(), df_class, 'edited', 'f1_micro', params=param_grid_svc,
                                     factor_cols=['code', 'cat'])
grid_svc_viewed_noreg = ModelEvaluator(SVC(), df_class, 'noticed', 'f1_micro', params=param_grid_svc,
                                       factor_cols=['code', 'cat'])

grid_svc_edits7sum_noreg.random_search(n_jobs=-1, n_iter=n_iter)
grid_svc_edits7sum_noreg.gs_res = grid_svc_edits7sum_noreg.rs_res
save_object(grid_svc_edits7sum_noreg, 'noreg_views_svc')

grid_svc_edits7sum_noreg.random_search(n_jobs=-1, n_iter=n_iter)
grid_svc_edits7sum_noreg.gs_res = grid_svc_edits7sum_noreg.rs_res
save_object(grid_svc_edits7sum_noreg, 'noreg_views_svc')

grid_svc_edit_noreg.random_search(n_jobs=-1, n_iter=n_iter)
grid_svc_edit_noreg.gs_res = grid_svc_edit_noreg.rs_res
save_object(grid_svc_edit_noreg, 'noreg_edited_svc')

grid_svc_viewed_noreg.random_search(n_jobs=-1, n_iter=n_iter)
grid_svc_viewed_noreg.gs_res = grid_svc_viewed_noreg.rs_res
save_object(grid_svc_viewed_noreg, 'noreg_viewed_svc')

2016-01-01 00:00:00 2019-01-01 00:00:00 2020-01-01 00:00:00
2016-01-01 00:00:00 2020-01-01 00:00:00 2021-01-01 00:00:00
2016-01-01 00:00:00 2019-01-01 00:00:00 2020-01-01 00:00:00
2016-01-01 00:00:00 2020-01-01 00:00:00 2021-01-01 00:00:00
2016-01-01 00:00:00 2019-01-01 00:00:00 2020-01-01 00:00:00
2016-01-01 00:00:00 2020-01-01 00:00:00 2021-01-01 00:00:00
2016-01-01 00:00:00 2019-01-01 00:00:00 2020-01-01 00:00:00
2016-01-01 00:00:00 2020-01-01 00:00:00 2021-01-01 00:00:00
